In [ ]:
from pandas import *
from datetime import datetime
import matplotlib.pyplot as plt

df = read_csv('Mortgage vs business vs all.csv')

## The column names are way too long, and the values are too high to quickly digest. Let's rename the columns and store the data in £billions, not just £.

MORTGAGE='Secured lending to individuals (£b)'
BUSINESS = 'Lending to non-financial businesses (£b)'

def milToBil (value):
    return value/1000

df[MORTGAGE] = df['Monthly amounts outstanding of monetary financial institutions sterling net secured lending to individuals (in sterling millions) seasonally adjusted'].apply(milToBil)

df[BUSINESS] = df['Monthly amounts outstanding of monetary financial institutions sterling net lending to private non-financial corporations (in sterling millions) seasonally adjusted'].apply(milToBil)

df = df[['Date', MORTGAGE, BUSINESS]]

## The data contain a column for each month, and I'm only interested in the yearly average. 
# Let's replace the 'Date' column with a 'Year' column, stored as Datetime, not a string.
# I'll strip the 'Date' column to its last two digits, then add the correct century. 
# This data covers the period 1998-2024, so we can find the century by the size of the number.

df['Year'] = df['Date'].str[-2:].astype(int)
df['Year'] = df['Year'].apply(lambda x: x + 2000 if x < 50 else x + 1900)
df = df[['Year',MORTGAGE,BUSINESS]]
df2 = df.groupby('Year').mean().reset_index()

## The data is still in reverse chronological order. Let's change that.

df2 = df2.iloc[::-1]


In [ ]:
## Now lets load and clean the GDP data.

gdp = read_excel('gdp uk.xlsx')
gdp2 = gdp[gdp['Year']>1997]
gdp2

def bil (value):
    return round((value/1000000000),2)

gdp2['gdp (£b)'] = gdp2['gdp (£ current)'].apply(bil)
gdp2 = gdp2[['gdp (£b)','Year']]


In [ ]:
## Lets merge the two data frames and express the lending as a percentage of GDP

MORTPC = 'Mortgage as % of GDP'
BUSPC = 'Non-financial lending as % of GDP'
df3 = merge(gdp2,df2,on=['Year'],how='outer')
df3[MORTPC] = (df3[MORTGAGE]/df3['gdp (£b)']) * 100
df3[BUSPC] = (df3[BUSINESS]/df3['gdp (£b)']) * 100
df4 = df3
df4.index = df4['Year']
df4[[MORTPC, BUSPC]].plot(grid=True, ylim=(0, 100), figsize=(8,5))


In [ ]:
## Now lets load and format the data for total M4ex, from the Bank of England

m4 = read_csv("M4ex 1998-2024.csv")
m4['M4 (£b)'] = m4['Monthly amounts outstanding of M4 (monetary financial institutions sterling M4 liabilities to private sector) (in sterling millions) seasonally adjusted'].apply(milToBil)
m4['Year'] = m4['Date'].str[-2:].astype(int)
m4['Year'] = m4['Year'].apply(lambda x: x + 2000 if x < 50 else x + 1900)
m4['Year'] = pd.to_datetime(m4['Year'], format='%Y')
m4 = m4[['Year','M4 (£b)']]
m4_annual = m4.groupby('Year').mean().reset_index()



In [ ]:
## Now lets merge it all and plot the value of mortgage vs all outstanding m4

df4 = df4[['gdp (£b)',MORTGAGE, BUSINESS, MORTPC, BUSPC]]
df4.reset_index()
all = merge(m4_annual, df4, on=('Year'), how='outer')
all.index = all['Year']
lendingM4Annual = all.groupby('Year').mean()
lendingM4Annual[[MORTGAGE, 'M4 (£b)']].plot(grid=True, figsize=(8,5))



In [ ]:
## Now lets load the house price:wage data from the ONS

houseWage = read_csv('House prices and wages Ingerland.csv')
houseWage['Year'] = houseWage['Year:']
houseWageMultiplier = HouseWage[['Year','House price:wage multiplier']]
houseWageMultiplier['Year'] = pd.to_datetime(houseWageMultiplier['Year'], format='%Y')
lendingm4Annual.reset_index()
all2 = merge(lendingM4Annual, houseWageMultiplier, on=('Year'), how='outer')
multiplier_vs_mortgage_pc = all2[['Year','Mortgage as % of GDP','House price:wage multiplier']]
multiplier_vs_mortgage_pc.index = multiplier_vs_mortgage_pc['Year']

## Now we need to plot the house price and the mortgage figures, using different axes

fig, ax1 = plt.subplots(figsize=(8,5))

ax1.set_xlabel('Year')
ax1.set_ylabel('Mortgage as % of GDP', color='tab:blue')
ax1.plot(multiplier_vs_mortgage_pc.index, multiplier_vs_mortgage_pc['Mortgage as % of GDP'], color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_ylim(0, 100) 

ax2 = ax1.twinx()
ax2.set_ylabel('House price:wage multiplier', color='tab:red')
ax2.plot(multiplier_vs_mortgage_pc.index, multiplier_vs_mortgage_pc['House price:wage multiplier'], color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.set_ylim(0, 10)

plt.title('Outstanding value of residential mortgages as % of GDP vs House price:wage multiplier, 1998-2022, England')

plt.show()